In [1]:
# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
import unicodedata
import spacy

from sklearn.manifold import TSNE
from contractions import CONTRACTION_MAP

#nlp = spacy.load('en_core', parse = True, tag=True, entity=True)
#nlp_vec = spacy.load('en_vecs', parse = True, tag=True, entity=True)
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#read the train and test file
train = pd.read_csv("train.csv",sep='~')
test = pd.read_csv("test.csv",sep='~')

In [3]:
#Encode the target variable
train = train[train['Is_Response'].notnull()]
stars_dict = {'Good':1,'Bad':0}
train["Is_Response"] = train['Is_Response'].replace(stars_dict,regex=True)
labels = train["Is_Response"]

In [4]:
#Remove ascented word
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [5]:
#Expand the contradictions
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [6]:
#Remove special character
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [7]:
train = train[train['Description'].notnull()]


In [8]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [9]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

Build a Text Normalizer

In [10]:
def normalize_corpus(corpus, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:

        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
            doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus

In [11]:
train['Description'] = normalize_corpus(train['Description'])
test['Description'] = normalize_corpus(test['Description'])

In [27]:
#Tokenization process
vocabulary_size = 50000
maxlen = 160
tokenizer = Tokenizer(num_words= vocabulary_size)
temp = pd.DataFrame()
temp['Description'] = train['Description']
temp = temp.append(test)
tokenizer.fit_on_texts(temp['Description'])

sequences = tokenizer.texts_to_sequences(train['Description'])
data = pad_sequences(sequences, maxlen=maxlen)
sequences = tokenizer.texts_to_sequences(test['Description'])
test_tokenize = pad_sequences(sequences, maxlen=maxlen)

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [28]:
print(temp.shape)

(38931, 4)


Build neural network with LSTM
Network Architechture
The network starts with an embedding layer. The layer lets the system expand each token to a more massive vector, allowing the network to represent a word in a meaningful way. The layer takes 50000 as the first argument, which is the size of our vocabulary, and 100 as the second input parameter, which is the dimension of the embeddings. The third parameter is the input_length of 50, which is the length of each comment sequence.

In [29]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=maxlen))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [30]:
model_conv = create_conv_model()
model_conv.fit(data, np.array(labels), validation_split=0.2, epochs = 3)

Train on 24136 samples, validate on 6035 samples
Epoch 1/3


 6496/24136 [=======>......................] - ETA: 20:22 - loss: 0.6953 - acc: 0.37 - ETA: 11:18 - loss: 0.6913 - acc: 0.50 - ETA: 8:15 - loss: 0.6858 - acc: 0.5625 - ETA: 6:45 - loss: 0.6776 - acc: 0.609 - ETA: 5:51 - loss: 0.6668 - acc: 0.650 - ETA: 5:15 - loss: 0.6662 - acc: 0.645 - ETA: 4:49 - loss: 0.6577 - acc: 0.660 - ETA: 4:29 - loss: 0.6561 - acc: 0.660 - ETA: 4:14 - loss: 0.6487 - acc: 0.666 - ETA: 4:02 - loss: 0.6484 - acc: 0.665 - ETA: 3:51 - loss: 0.6507 - acc: 0.661 - ETA: 3:43 - loss: 0.6422 - acc: 0.669 - ETA: 3:36 - loss: 0.6407 - acc: 0.668 - ETA: 3:30 - loss: 0.6411 - acc: 0.667 - ETA: 3:25 - loss: 0.6439 - acc: 0.664 - ETA: 3:20 - loss: 0.6363 - acc: 0.671 - ETA: 3:15 - loss: 0.6354 - acc: 0.671 - ETA: 3:12 - loss: 0.6360 - acc: 0.668 - ETA: 3:08 - loss: 0.6371 - acc: 0.666 - ETA: 3:05 - loss: 0.6362 - acc: 0.664 - ETA: 3:02 - loss: 0.6322 - acc: 0.669 - ETA: 3:00 - loss: 0.6344 - acc: 0.667 - ETA: 2:57 - loss: 0.6345 - acc: 0.665 - ETA: 2:55 - loss: 0.6331 - acc: 

13024/24136 [===============>..............] - ETA: 1:40 - loss: 0.4504 - acc: 0.787 - ETA: 1:40 - loss: 0.4515 - acc: 0.786 - ETA: 1:40 - loss: 0.4520 - acc: 0.786 - ETA: 1:40 - loss: 0.4513 - acc: 0.786 - ETA: 1:40 - loss: 0.4507 - acc: 0.787 - ETA: 1:39 - loss: 0.4501 - acc: 0.787 - ETA: 1:39 - loss: 0.4492 - acc: 0.788 - ETA: 1:39 - loss: 0.4484 - acc: 0.788 - ETA: 1:39 - loss: 0.4482 - acc: 0.788 - ETA: 1:38 - loss: 0.4476 - acc: 0.788 - ETA: 1:38 - loss: 0.4470 - acc: 0.789 - ETA: 1:38 - loss: 0.4461 - acc: 0.790 - ETA: 1:38 - loss: 0.4451 - acc: 0.790 - ETA: 1:38 - loss: 0.4442 - acc: 0.790 - ETA: 1:37 - loss: 0.4437 - acc: 0.791 - ETA: 1:37 - loss: 0.4440 - acc: 0.791 - ETA: 1:37 - loss: 0.4436 - acc: 0.791 - ETA: 1:37 - loss: 0.4428 - acc: 0.791 - ETA: 1:36 - loss: 0.4417 - acc: 0.792 - ETA: 1:36 - loss: 0.4409 - acc: 0.793 - ETA: 1:36 - loss: 0.4402 - acc: 0.793 - ETA: 1:36 - loss: 0.4399 - acc: 0.793 - ETA: 1:36 - loss: 0.4393 - acc: 0.794 - ETA: 1:35 - loss: 0.4389 - acc: 0

19872/24136 [=======================>......] - ETA: 1:01 - loss: 0.3832 - acc: 0.826 - ETA: 1:00 - loss: 0.3829 - acc: 0.826 - ETA: 1:00 - loss: 0.3827 - acc: 0.826 - ETA: 1:00 - loss: 0.3825 - acc: 0.826 - ETA: 1:00 - loss: 0.3828 - acc: 0.826 - ETA: 1:00 - loss: 0.3828 - acc: 0.827 - ETA: 1:00 - loss: 0.3827 - acc: 0.827 - ETA: 59s - loss: 0.3831 - acc: 0.827 - ETA: 59s - loss: 0.3829 - acc: 0.82 - ETA: 59s - loss: 0.3825 - acc: 0.82 - ETA: 59s - loss: 0.3822 - acc: 0.82 - ETA: 59s - loss: 0.3821 - acc: 0.82 - ETA: 59s - loss: 0.3821 - acc: 0.82 - ETA: 58s - loss: 0.3822 - acc: 0.82 - ETA: 58s - loss: 0.3818 - acc: 0.82 - ETA: 58s - loss: 0.3821 - acc: 0.82 - ETA: 58s - loss: 0.3818 - acc: 0.82 - ETA: 58s - loss: 0.3815 - acc: 0.82 - ETA: 57s - loss: 0.3812 - acc: 0.82 - ETA: 57s - loss: 0.3812 - acc: 0.82 - ETA: 57s - loss: 0.3810 - acc: 0.82 - ETA: 57s - loss: 0.3811 - acc: 0.82 - ETA: 57s - loss: 0.3808 - acc: 0.82 - ETA: 57s - loss: 0.3807 - acc: 0.82 - ETA: 56s - loss: 0.3804 - 

24136/24136 [==============================] - ETA: 23s - loss: 0.3562 - acc: 0.84 - ETA: 22s - loss: 0.3562 - acc: 0.84 - ETA: 22s - loss: 0.3563 - acc: 0.84 - ETA: 22s - loss: 0.3561 - acc: 0.84 - ETA: 22s - loss: 0.3562 - acc: 0.84 - ETA: 22s - loss: 0.3560 - acc: 0.84 - ETA: 22s - loss: 0.3557 - acc: 0.84 - ETA: 21s - loss: 0.3556 - acc: 0.84 - ETA: 21s - loss: 0.3552 - acc: 0.84 - ETA: 21s - loss: 0.3551 - acc: 0.84 - ETA: 21s - loss: 0.3551 - acc: 0.84 - ETA: 21s - loss: 0.3551 - acc: 0.84 - ETA: 20s - loss: 0.3551 - acc: 0.84 - ETA: 20s - loss: 0.3550 - acc: 0.84 - ETA: 20s - loss: 0.3550 - acc: 0.84 - ETA: 20s - loss: 0.3548 - acc: 0.84 - ETA: 20s - loss: 0.3548 - acc: 0.84 - ETA: 20s - loss: 0.3548 - acc: 0.84 - ETA: 19s - loss: 0.3547 - acc: 0.84 - ETA: 19s - loss: 0.3545 - acc: 0.84 - ETA: 19s - loss: 0.3542 - acc: 0.84 - ETA: 19s - loss: 0.3540 - acc: 0.84 - ETA: 19s - loss: 0.3539 - acc: 0.84 - ETA: 19s - loss: 0.3537 - acc: 0.84 - ETA: 18s - loss: 0.3538 - acc: 0.84 - ETA

...

24136/24136 [==============================] - ETA: 22s - loss: 0.2027 - acc: 0.92 - ETA: 22s - loss: 0.2026 - acc: 0.92 - ETA: 21s - loss: 0.2027 - acc: 0.92 - ETA: 21s - loss: 0.2027 - acc: 0.92 - ETA: 21s - loss: 0.2032 - acc: 0.92 - ETA: 21s - loss: 0.2033 - acc: 0.92 - ETA: 21s - loss: 0.2033 - acc: 0.92 - ETA: 21s - loss: 0.2032 - acc: 0.92 - ETA: 20s - loss: 0.2033 - acc: 0.92 - ETA: 20s - loss: 0.2034 - acc: 0.92 - ETA: 20s - loss: 0.2033 - acc: 0.92 - ETA: 20s - loss: 0.2036 - acc: 0.92 - ETA: 20s - loss: 0.2034 - acc: 0.92 - ETA: 20s - loss: 0.2034 - acc: 0.92 - ETA: 19s - loss: 0.2034 - acc: 0.92 - ETA: 19s - loss: 0.2032 - acc: 0.92 - ETA: 19s - loss: 0.2032 - acc: 0.92 - ETA: 19s - loss: 0.2032 - acc: 0.92 - ETA: 19s - loss: 0.2030 - acc: 0.92 - ETA: 19s - loss: 0.2030 - acc: 0.92 - ETA: 18s - loss: 0.2029 - acc: 0.92 - ETA: 18s - loss: 0.2032 - acc: 0.92 - ETA: 18s - loss: 0.2033 - acc: 0.92 - ETA: 18s - loss: 0.2032 - acc: 0.92 - ETA: 18s - loss: 0.2034 - acc: 0.92 - ETA

 6528/24136 [=======>......................] - ETA: 2:09 - loss: 0.1627 - acc: 0.968 - ETA: 2:09 - loss: 0.1034 - acc: 0.984 - ETA: 2:05 - loss: 0.1297 - acc: 0.979 - ETA: 2:06 - loss: 0.1087 - acc: 0.984 - ETA: 2:07 - loss: 0.0939 - acc: 0.987 - ETA: 2:06 - loss: 0.0867 - acc: 0.989 - ETA: 2:06 - loss: 0.0954 - acc: 0.982 - ETA: 2:06 - loss: 0.0903 - acc: 0.984 - ETA: 2:06 - loss: 0.0930 - acc: 0.982 - ETA: 2:06 - loss: 0.0922 - acc: 0.981 - ETA: 2:06 - loss: 0.0891 - acc: 0.983 - ETA: 2:06 - loss: 0.0937 - acc: 0.979 - ETA: 2:06 - loss: 0.0885 - acc: 0.980 - ETA: 2:06 - loss: 0.0937 - acc: 0.979 - ETA: 2:06 - loss: 0.0913 - acc: 0.981 - ETA: 2:05 - loss: 0.0985 - acc: 0.976 - ETA: 2:06 - loss: 0.0972 - acc: 0.976 - ETA: 2:05 - loss: 0.1027 - acc: 0.974 - ETA: 2:05 - loss: 0.1009 - acc: 0.975 - ETA: 2:05 - loss: 0.0986 - acc: 0.976 - ETA: 2:04 - loss: 0.0978 - acc: 0.976 - ETA: 2:04 - loss: 0.1008 - acc: 0.974 - ETA: 2:04 - loss: 0.0980 - acc: 0.975 - ETA: 2:04 - loss: 0.1053 - acc: 0

13088/24136 [===============>..............] - ETA: 1:33 - loss: 0.1055 - acc: 0.964 - ETA: 1:32 - loss: 0.1056 - acc: 0.964 - ETA: 1:32 - loss: 0.1060 - acc: 0.963 - ETA: 1:32 - loss: 0.1061 - acc: 0.963 - ETA: 1:32 - loss: 0.1057 - acc: 0.963 - ETA: 1:32 - loss: 0.1054 - acc: 0.964 - ETA: 1:32 - loss: 0.1051 - acc: 0.964 - ETA: 1:31 - loss: 0.1052 - acc: 0.964 - ETA: 1:31 - loss: 0.1049 - acc: 0.964 - ETA: 1:31 - loss: 0.1054 - acc: 0.963 - ETA: 1:31 - loss: 0.1053 - acc: 0.964 - ETA: 1:31 - loss: 0.1058 - acc: 0.963 - ETA: 1:31 - loss: 0.1056 - acc: 0.964 - ETA: 1:30 - loss: 0.1054 - acc: 0.964 - ETA: 1:30 - loss: 0.1050 - acc: 0.964 - ETA: 1:30 - loss: 0.1054 - acc: 0.964 - ETA: 1:30 - loss: 0.1054 - acc: 0.964 - ETA: 1:30 - loss: 0.1052 - acc: 0.964 - ETA: 1:30 - loss: 0.1049 - acc: 0.964 - ETA: 1:29 - loss: 0.1048 - acc: 0.964 - ETA: 1:29 - loss: 0.1046 - acc: 0.964 - ETA: 1:29 - loss: 0.1044 - acc: 0.964 - ETA: 1:29 - loss: 0.1046 - acc: 0.964 - ETA: 1:29 - loss: 0.1044 - acc: 0

19968/24136 [=======================>......] - ETA: 58s - loss: 0.1037 - acc: 0.96 - ETA: 58s - loss: 0.1037 - acc: 0.96 - ETA: 58s - loss: 0.1036 - acc: 0.96 - ETA: 57s - loss: 0.1034 - acc: 0.96 - ETA: 57s - loss: 0.1037 - acc: 0.96 - ETA: 57s - loss: 0.1036 - acc: 0.96 - ETA: 57s - loss: 0.1034 - acc: 0.96 - ETA: 57s - loss: 0.1034 - acc: 0.96 - ETA: 56s - loss: 0.1033 - acc: 0.96 - ETA: 56s - loss: 0.1037 - acc: 0.96 - ETA: 56s - loss: 0.1036 - acc: 0.96 - ETA: 56s - loss: 0.1038 - acc: 0.96 - ETA: 56s - loss: 0.1037 - acc: 0.96 - ETA: 56s - loss: 0.1035 - acc: 0.96 - ETA: 55s - loss: 0.1034 - acc: 0.96 - ETA: 55s - loss: 0.1037 - acc: 0.96 - ETA: 55s - loss: 0.1035 - acc: 0.96 - ETA: 55s - loss: 0.1041 - acc: 0.96 - ETA: 55s - loss: 0.1040 - acc: 0.96 - ETA: 55s - loss: 0.1039 - acc: 0.96 - ETA: 54s - loss: 0.1037 - acc: 0.96 - ETA: 54s - loss: 0.1045 - acc: 0.96 - ETA: 54s - loss: 0.1045 - acc: 0.96 - ETA: 54s - loss: 0.1044 - acc: 0.96 - ETA: 54s - loss: 0.1043 - acc: 0.96 - ETA

24136/24136 [==============================] - ETA: 21s - loss: 0.1102 - acc: 0.96 - ETA: 21s - loss: 0.1102 - acc: 0.96 - ETA: 21s - loss: 0.1102 - acc: 0.96 - ETA: 21s - loss: 0.1101 - acc: 0.96 - ETA: 21s - loss: 0.1102 - acc: 0.96 - ETA: 21s - loss: 0.1103 - acc: 0.96 - ETA: 20s - loss: 0.1105 - acc: 0.96 - ETA: 20s - loss: 0.1104 - acc: 0.96 - ETA: 20s - loss: 0.1105 - acc: 0.96 - ETA: 20s - loss: 0.1104 - acc: 0.96 - ETA: 20s - loss: 0.1103 - acc: 0.96 - ETA: 20s - loss: 0.1103 - acc: 0.96 - ETA: 19s - loss: 0.1103 - acc: 0.96 - ETA: 19s - loss: 0.1104 - acc: 0.96 - ETA: 19s - loss: 0.1105 - acc: 0.96 - ETA: 19s - loss: 0.1105 - acc: 0.96 - ETA: 19s - loss: 0.1104 - acc: 0.96 - ETA: 19s - loss: 0.1106 - acc: 0.96 - ETA: 18s - loss: 0.1105 - acc: 0.96 - ETA: 18s - loss: 0.1105 - acc: 0.96 - ETA: 18s - loss: 0.1103 - acc: 0.96 - ETA: 18s - loss: 0.1103 - acc: 0.96 - ETA: 18s - loss: 0.1104 - acc: 0.96 - ETA: 18s - loss: 0.1106 - acc: 0.96 - ETA: 17s - loss: 0.1107 - acc: 0.96 - ETA

In [36]:
predictions = model_conv.predict_classes(test_tokenize)

In [39]:
out_df = pd.DataFrame({"User_ID":test["User_ID"].values})
out_df['Is_Response'] = predictions
stars_dict = {1:'Good',0:'Bad'}
out_df["Is_Response"] = out_df['Is_Response'].replace(stars_dict,regex=True)
out_df.to_csv("submission_LSTM.csv", index=False)
out_df.head()

,User_ID,Is_Response
0,9602,Good
1,8749,Good
2,15500,Good
3,5495,Good
4,18570,Good
